In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'/Users/cameronpaschall/Desktop/Instacart Basket Analysis'

In [3]:
# Import data set
cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

## Wrangling Data

In [4]:
cust.columns

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [5]:
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [6]:
# Rename columns
cust = cust.rename(columns = {'First Name':'first_name', 'Surnam':'last_name', 'Gender':'gender', 'STATE':'state', 'Age':'age'})

In [7]:
# check for mixed types

for col in cust.columns.tolist():
  weird = (cust[[col]].applymap(type) != cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (cust[weird]) > 0:
    print (col)

first_name


In [8]:
# fix first name column

cust['first_name'] = cust['first_name'].astype('str')

## Step 5 further data checks

In [9]:
cust.dtypes

user_id          int64
first_name      object
last_name       object
gender          object
state           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [10]:
# check for duplicates
dups = cust[cust.duplicated()]

In [11]:
dups

Empty DataFrame
Columns: [user_id, first_name, last_name, gender, state, age, date_joined, n_dependants, fam_status, income]
Index: []

In [12]:
# check for missing values

cust.isnull().sum()

user_id         0
first_name      0
last_name       0
gender          0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

## Step 6 combine data sets

In [13]:
# Import merged dataframe

opm = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_flags.pkl'))

In [14]:
cust.head(5)

user_id first_name last_name  gender       state  age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

In [15]:
opm.head(5)

order_id  user_id  order_number  orders_day_of_week  order_time_of_day  \
0   2539329        1             1                   2                  8   
1   2539329        1             1                   2                  8   
2   2539329        1             1                   2                  8   
3   2539329        1             1                   2                  8   
4   2539329        1             1                   2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                    1.11         196                  1          0   
1                    1.11       14084                  2          0   
2                    1.11       12427                  3          0   
3                    1.11       26088                  4          0   
4                    1.11       26405                  5          0   

                              product_name  ...    price_range_loc  \
0                                     Soda  ...  Mid-range product   
1  Organic Unsweetened Vanilla Almond Milk  ...  Mid-range product   
2                      Original Beef Jerky  ...  Low-range product   
3               Aged White Cheddar Popcorn  ...  Low-range product   
4         XL Pick-A-Size Paper Towel Rolls  ...  Low-range product   

     Busiest days    Slowest days busiest_period_of_day max_order  \
0  Regularly busy  Regularly busy        Average orders        10   
1  Regularly busy  Regularly busy        Average orders        10   
2  Regularly busy  Regularly busy        Average orders        10   
3  Regularly busy  Regularly busy        Average orders        10   
4  Regularly busy  Regularly busy        Average orders        10   

   loyalty_flag  spending type_of_spender  order_freq    frequency_flag  
0  New customer  6.367797     Low Spender        20.0  Regular Customer  
1  New customer  6.367797     Low Spender        20.0  Regular Customer  
2  New customer  6.367797     Low Spender        20.0  Regular Customer  
3  New customer  6.367797     Low Spender        20.0  Regular Customer  
4  New customer  6.367797     Low Spender        20.0  Regular Customer  

[5 rows x 24 columns]

In [16]:
# check user id data type
cust['user_id'].dtype

dtype('int64')

In [17]:
# check user id data type
opm['user_id'].dtype

dtype('int64')

In [18]:
# merging dataframes

df_ords_prods_cust = opm.merge(cust, on = 'user_id', indicator = 'merge')

In [19]:
df_ords_prods_cust.head()

order_id  user_id  order_number  orders_day_of_week  order_time_of_day  \
0   2539329        1             1                   2                  8   
1   2539329        1             1                   2                  8   
2   2539329        1             1                   2                  8   
3   2539329        1             1                   2                  8   
4   2539329        1             1                   2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                    1.11         196                  1          0   
1                    1.11       14084                  2          0   
2                    1.11       12427                  3          0   
3                    1.11       26088                  4          0   
4                    1.11       26405                  5          0   

                              product_name  ...  first_name  last_name  \
0                                     Soda  ...       Linda     Nguyen   
1  Organic Unsweetened Vanilla Almond Milk  ...       Linda     Nguyen   
2                      Original Beef Jerky  ...       Linda     Nguyen   
3               Aged White Cheddar Popcorn  ...       Linda     Nguyen   
4         XL Pick-A-Size Paper Towel Rolls  ...       Linda     Nguyen   

   gender    state age date_joined n_dependants fam_status  income merge  
0  Female  Alabama  31   2/17/2019            3    married   40423  both  
1  Female  Alabama  31   2/17/2019            3    married   40423  both  
2  Female  Alabama  31   2/17/2019            3    married   40423  both  
3  Female  Alabama  31   2/17/2019            3    married   40423  both  
4  Female  Alabama  31   2/17/2019            3    married   40423  both  

[5 rows x 34 columns]

## Step 8 Export new dataframe

In [20]:
df_ords_prods_cust.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_ords_prods_cust.pkl'))